### import library

In [1]:
import re
from datetime import datetime
import os
import numpy as np
import pandas as pd
from time import time, sleep
from selenium import webdriver
import requests as r
from IPython.core.display import clear_output
from warnings import warn
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [69]:
KEYWORD = "tabrak"
LOCATION = "bromo"

STR = '{keyword}_radar{location}.jawapos.com_1-1-2020_12-30-2020.csv'
FILENAME = STR.format(keyword = KEYWORD, location = LOCATION)

FOLDER_DIR = os.path.join(os.getcwd(),'googlenews')
FILE_DIR = os.path.join(FOLDER_DIR,FILENAME)

data = pd.read_csv(FILE_DIR).iloc[:,1:]
links = np.asarray(data.iloc[:,0])
len(links)

300

In [70]:
def convert_link(links):
    links_new = []
    for link in links:
        l = link.split('/')
        links_new.append('/'.join(l[0:len(l)-1]))
    return links_new
links = convert_link(links)

In [72]:
path = r"C:\Users\Dwi Nissa\AppData\Local\Programs\Python\Python38\Lib\site-packages\selenium\webdriver\firefox\geckodriver.exe"
options = webdriver.FirefoxOptions()
options.set_preference("dom.push.enabled", False)
driver = webdriver.Firefox(executable_path = path, firefox_options=options)

news = []
start_time = time()
req = 0;
a = 0;

clear_output(wait = True);
for i in range(len(links)):
    content = ""; datetime = ""; category = "" ; title = ""
    try:
        #print(links[i])
        driver.get(links[i])
        sleep(2)
        
        # log info
        req+=1 ; a+=1
        elapsed_time = time() - start_time
        print("Request:{}; Frequency:{} req/sec".format(req, req/elapsed_time))
        if a>9:
            clear_output(wait = True);
            a = 0;
        # scrap the content using one of this syntax! 
        # InshaAllah terambil, karna udah disesuaikan untuk semua radar2 di Jawa Timur hehe
        try:
            content = driver.find_elements(By.XPATH,"//div[@class='tdb-block-inner td-fix-index']")[15].text.strip()
        except(NoSuchElementException, IndexError, ValueError):
            pass
        if content=="":
            try:
                content = driver.find_element_by_xpath("//div[@itemprop='articleBody']").text.strip()
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                content = driver.find_element_by_xpath("//div[@dir='auto']").text.strip()
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                content = driver.find_element_by_xpath("//div[@class='entry-content entry clearfix']").text.strip()
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                contents = [con.text for cont in driver.find_elements_by_xpath("//div/p[@style='text-align: justify']")]
                content = ' '.join(contents).strip()
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                contents = [cont.text for cont in driver.find_elements_by_xpath("//div[@class='paragraph-article-detail']/p")]
                content = ' '.join(contents).strip()
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if  len(content)<500:
            try:
                contents = [cont.text for cont in driver.find_elements_by_xpath("//div[@class='tdb-block-inner td-fix-index']/p")]
                content = ' '.join(contents).strip()
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                contents = [cont.text for cont in driver.find_elements_by_xpath("//div[@class='content-inner']/div[2]/p")]
                content = ' '.join(contents).strip()
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                contents = [cont.text for cont in driver.find_elements_by_xpath("//div[@class='content-inner  mobile-truncate']/div[2]/p")]
                content = ' '.join(contents).strip()
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                content = driver.find_element_by_css_selector(".td-post-content").text.strip()
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                content=' '.join([par.text.strip() for par in driver.find_elements_by_xpath("//div[@class='content-inner']/p")])
            except(NoSuchElementException, IndexError, ValueError):
                pass
        try:
            datetime = driver.find_element_by_css_selector(".tdb_single_date>div:nth-child(3)>time:nth-child(1)").text.strip()
        except(NoSuchElementException, IndexError, ValueError):
            pass
        try:
            category = driver.find_element_by_css_selector(".tdb-entry-category").text.strip()
        except(NoSuchElementException, IndexError, ValueError):
            pass
        try:
            title = driver.find_element_by_css_selector(".tdb-title-text").text.strip()
        except(NoSuchElementException, IndexError, ValueError):
            pass
        news.append([links[i],datetime,category,title,content])
    except:
        pass

print('-----------------------FINISHED-----------------------')
print('--------Scrap Duration: {} minute(s).--------'.format((time()-start_time)/60))
data = pd.DataFrame(news, columns=['Link','Date Time','Category','Title','Content'])
data

Request:291; Frequency:0.05348767454360046 req/sec
Request:292; Frequency:0.05346269285855319 req/sec
Request:293; Frequency:0.05045680326734654 req/sec
Request:294; Frequency:0.050430755531443903 req/sec
Request:295; Frequency:0.0503352925905043 req/sec
Request:296; Frequency:0.050383018495700746 req/sec
Request:297; Frequency:0.049972940107765894 req/sec
Request:298; Frequency:0.04970186140796921 req/sec
-----------------------FINISHED-----------------------
--------Scrap Duration: 99.93015304406484 minute(s).--------


,Link,Date Time,Category,Title,Content
0,https://radarbromo.jawapos.com/news/24/12/2020...,"Thursday, 24 December 2020",NEWS,Pesimistis Akan Ada Suplai Ternak di Balai Kar...,"KRAKSAAN, Radar Bromo – Dari awal pandemi Covi..."
1,https://radarbromo.jawapos.com/news/23/12/2020...,"Wednesday, 23 December 2020",NEWS,"Mau Kerja, Pegawai Dishub Tewas Jadi Korban Ta...","BEJI, Radar Bromo– Nahas nian nasib yang diala..."
2,https://radarbromo.jawapos.com/news/23/12/2020...,"Wednesday, 23 December 2020",NEWS,Sediakan Toilet Portable di Exit Tol Leces,"KRAKSAAN, Radar Bromo – Beragam persiapan dila..."
3,https://radarbromo.jawapos.com/pasuruan/22/12/...,"Tuesday, 22 December 2020",PASURUAN,Kapolres Pasuruan: Tahun Baru Jangan Sampai Ge...,"BANGIL, Radar Bromo – Momen liburan Natal dan ..."
4,https://radarbromo.jawapos.com/pandaan/22/12/2...,"Tuesday, 22 December 2020",PANDAAN,"1st Anniversary, Pandul Gelar Gowes Penghijaua...",KOMUNITAS Pandaan Peduli (Pandul) punya cara t...
...,...,...,...,...,...
293,https://radarbromo.jawapos.com/pasuruan/20/11/...,"Friday, 20 November 2020",PASURUAN,"Usulkan Relokasi Warga di Lahan TNI AL, Bisa M...","BANGIL, Radar Bromo – Pembahasan perubahan Per..."
294,https://radarbromo.jawapos.com/hukrim/20/11/20...,"Friday, 20 November 2020",HUKUM DAN KRIMINAL,Dua Pelaku Curi Motor di Alfamart Jl Gubernur ...,"KANIGARAN, Radar Bromo – Sebuah aksi pencurian..."
295,https://radarbromo.jawapos.com/hukrim/20/11/20...,"Friday, 20 November 2020",HUKUM DAN KRIMINAL,Nama Jon Kembali Muncul saat Kadir Jadi Saksi ...,"KRAKSAAN, Radar Bromo – Abdul Kadir mantan ter..."
296,https://radarbromo.jawapos.com/kraksaan/18/11/...,"Wednesday, 18 November 2020",KRAKSAAN,Operasi Masker Kembali Digalakkan di Kab Probo...,"MARON, Radar Bromo – Hingga saat ini masih dit..."


In [73]:
OUT_GOOGLE_DIR = os.path.join(os.getcwd(), 'googlenews_out')
OUT_DIR = os.path.join(OUT_GOOGLE_DIR, "radar{}".format(LOCATION))
if not os.path.exists(OUT_GOOGLE_DIR):
    os.mkdir(OUT_GOOGLE_DIR)
if not os.path.exists(OUT_DIR):
        os.mkdir(OUT_DIR)
OUT_FILENAME = os.path.join(OUT_DIR,'radar{}_{}.csv'.format(LOCATION,KEYWORD))
data.to_csv(OUT_FILENAME)
print('File saved as {}'.format(OUT_FILENAME))

File saved as C:\Users\Dwi Nissa\Skripsi\CARI METODE\SCRAPPING\googlenews_out\radarbromo\radarbromo_tabrak.csv
